# **Machine Learning App in R**

## **Brian J. Ault**

### **Load The Data**

In [2]:
# attach the iris dataset to the environment
data(iris)
# rename the dataset
dataset <- iris

In [3]:
# define the filename
filename <- "iris.csv"
# load the CSV file from the local directory
dataset <- read.csv(filename, header=FALSE)
# set the column names in the dataset
colnames(dataset) <- c("Sepal.Length","Sepal.Width","Petal.Length","Petal.Width","Species")

### **Create a Validation Dataset**

In [5]:
# install.packages("caret")
library(caret)

# Assuming 'dataset' is your original dataset

# Set seed for reproducibility
set.seed(123)

# Create an index to split the data
validation_index <- createDataPartition(dataset$Species, p = 0.80, list = FALSE)

# Create training and validation datasets
training <- dataset[validation_index, ]  # 80% of original dataset
validation <- dataset[-validation_index, ]  # 20% of original dataset

ERROR: Error in library(caret): there is no package called ‘caret’
